In [262]:
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix
import numpy as np
from glob import glob
from cv2 import aruco
import cv2
import time

In [263]:
n_cameras = 1
camera_params = np.array([[4,2104,1560]])

In [264]:
from random import random


bilder = glob('../bildverband1/*.jpg')
n_images = len(bilder)
image_params = np.empty(n_images*6)
image_params.fill(0)
 
for i in range(n_images):
    image_params[i*6]=0
    image_params[i*6+1]=0
    image_params[i*6+2]=0
    image_params[i*6+3]=100+(random()-0.5)*10
    image_params[i*6+4] = 100+(random()-0.5)*10
    image_params[i*6+5]=100+(random()-0.5)*1

In [265]:
LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

aruco_dict = aruco.Dictionary_create(32, 3)
n_observations = 0

camera_indices = np.array([], dtype=int)
image_indices = np.array([], dtype=int)
point_indices = np.array([], dtype=int)
points_2d_list = []

markers = []

for bildnr in range(len(bilder)):
    bild = bilder[bildnr]
    cv_img = cv2.imread(bild)
    tmp = cv2.LUT(cv_img, lut)
    gray = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray,
                                          aruco_dict)

    for nr in range(len(ids)):
        for cid in range(len(corners[nr][0])):
            if not (ids[nr][0]*10 + cid) in markers:
                markers.append(ids[nr][0]*10 + cid)
            pid = markers.index(ids[nr][0]*10 + cid)
            point_indices = np.append(point_indices, pid)
            image_indices = np.append(image_indices, bildnr)
            camera_indices = np.append(camera_indices, 0)
            points_2d_list.append(corners[nr][0][cid])
            n_observations += 1

n_points = len(markers)
points_3d = np.empty(n_points * 3)
points_3d.fill(0)
points_2d = np.array(points_2d_list)

for i in range(n_points):
    points_3d[i*3+0] = 100+(random()-0.5)*10
    points_3d[i*3+1] = 100+(random()-0.5)*10
    points_3d[i*3+2] = 95+(random()-0.5)*1


In [266]:
points_2d

array([[2476., 2227.],
       [2381., 2133.],
       [2473., 2003.],
       [2566., 2101.],
       [2946., 1516.],
       [3031., 1601.],
       [2960., 1716.],
       [2876., 1632.],
       [1541., 2676.],
       [1661., 2513.],
       [1787., 2612.],
       [1671., 2774.],
       [1861., 2021.],
       [2013., 1960.],
       [2052., 2121.],
       [1903., 2187.],
       [2131., 1187.],
       [2270., 1146.],
       [2311., 1307.],
       [2177., 1351.],
       [1207.,  864.],
       [1412.,  851.],
       [1447., 1049.],
       [1247., 1069.],
       [2805.,  839.],
       [2903.,  916.],
       [2852., 1047.],
       [2749.,  972.],
       [2109., 1949.],
       [2075., 1750.],
       [2356., 1707.],
       [2415., 1896.],
       [ 647., 1560.],
       [ 914., 1525.],
       [ 856., 1687.],
       [ 575., 1723.],
       [1592., 1335.],
       [1797., 1372.],
       [1673., 1509.],
       [1459., 1463.],
       [3697., 1916.],
       [3570., 1984.],
       [3448., 1798.],
       [357

In [267]:
n_observations

96

In [268]:
def rotate(points, rot_vecs):
    """Rotate points by given rotation vectors.
    
    Rodrigues' rotation formula is used.
    :param rot_vecs: [r1,r2,r3]
    """
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v


In [269]:
def project(points, camera_params, image_params):
    """
    Convert 3-D points to 2-D by projecting onto images.
    :param camera_params: [[f,k1,k2]]
    :param image_params: [[camera_indices, r1,r2,r3,x0,y0,z0]]
    """
    points_proj = rotate(points, image_params[:, 1:4])
    points_proj += image_params[:, 3:]
    points_proj = -points_proj[:, :2] / points_proj[:, 2, np.newaxis]
    f = camera_params[:,0]
    k1 = camera_params[:,1]
    k2 = camera_params[:,2]
    n = np.sum(points_proj**2, axis=1)
    r = 1 + k1 * n + k2 * n**2
    points_proj *= (r * f)[:, np.newaxis]
    return points_proj


In [270]:
def fun(params, n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d):
    """Compute residuals.
    
    `params` contains camera parameters and 3-D coordinates.
    """
    camera_params = params[:n_cameras * 3].reshape((n_cameras, 3))
    image_params = params[n_cameras * 3:n_cameras *
                          3 + n_images * 6].reshape((n_images, 6))
    points_3d = params[n_cameras * 3 + n_images * 6:].reshape((n_points, 3))
    points_proj = project(
        points_3d[point_indices], camera_params[camera_indices], image_params[image_indices])
    return (points_proj - points_2d).ravel()


In [271]:
def bundle_adjustment_sparsity(n_cameras, n_images, n_points, camera_indices, images_indices, point_indices):
    m = camera_indices.size * 2
    n = n_cameras * 3 + n_images * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)
    for s in range(9):
        A[2 * i, camera_indices * 9 + s] = 1
        A[2 * i + 1, camera_indices * 9 + s] = 1

    for s in range(3):
        A[2 * i, n_cameras * 9 + point_indices * 3 + s] = 1
        A[2 * i + 1, n_cameras * 9 + point_indices * 3 + s] = 1

    return A


In [272]:
x0 = np.hstack((camera_params.ravel(), image_params.ravel(), points_3d.ravel()))
f0 = fun(x0, n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d)

In [273]:
A = bundle_adjustment_sparsity(
    n_cameras, n_images, n_points, camera_indices, image_indices, point_indices)
t0 = time.time()
res = least_squares(fun, x0, jac_sparsity=A, verbose=2, x_scale='jac', ftol=1e-4, method='trf',
                    args=(n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d))
t1 = time.time()

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.3205e+10                                    2.11e+11    
       1              3         3.6145e+09      9.59e+09       1.30e+03       2.96e+10    
       2              4         5.7840e+08      3.04e+09       1.25e+03       3.00e+09    
       3              6         4.1445e+08      1.64e+08       4.17e+02       5.56e+08    
       4              9         4.0641e+08      8.04e+06       4.60e+01       4.78e+08    
       5             10         4.0391e+08      2.50e+06       3.44e+01       4.31e+08    
       6             12         4.0377e+08      1.36e+05       5.39e+00       4.27e+08    
       7             22         4.0377e+08      0.00e+00       0.00e+00       4.27e+08    
`xtol` termination condition is satisfied.
Function evaluations 22, initial cost 1.3205e+10, final cost 4.0377e+08, first-order optimality 4.27e+08.


In [277]:
res.x[0:n_cameras*3].reshape(n_cameras, 3)

array([[   2.33995616,  576.37501755, -234.30718931]])

In [284]:

for i in res.x[n_cameras*3:n_cameras*3+n_images*6].reshape(n_images, 6)[:, 3:]:
    print('v', i[0], i[1], i[2])


v 95.73091026686288 -21.526421336239082 -364.26718824319016
v 101.58974814774737 94.93468883766498 134.25458300508265
v 103.25133961066658 143.40311049273066 100.60536340193154
v 98.40926321011867 131.74149812445395 193.78472471883651


In [282]:
for i in res.x[n_cameras*3+n_images*6:].reshape(n_points, 3).reshape(n_points, 3):
    print ('v',i[0],i[1],i[2])


v 102.16285870263184 284.8868746823323 136.34847886296683
v 103.48368802096955 122.72338523878258 61.19322268478749
v 104.04058142799182 129.73443338338552 94.92509008301043
v 98.47914487342605 109.17099205929237 94.81847660798196
v 97.25473466873594 160.9124386026953 94.85003225489953
v 161.50908129712408 102.02500130109608 95.35579596077878
v 96.41118756217058 105.29841338285506 95.09139566649905
v 96.2674625017517 96.08386579449586 104.79242413164025
v 98.20853449129908 148.83418497840304 95.3144097534422
v 101.8249049865935 100.54685629339022 131.98321216520623
v 89.82014251260622 168.2963065989634 94.64530198037944
v -0.31047635102808624 98.32545840468512 101.43850537450072
v 97.12216357421138 158.54761846267166 118.578801729345
v 105.02360055159697 251.47850999528234 42.74655709185787
v 84.4137469163593 290.3968486888971 95.45062391567795
v 149.9523388071215 87.14216122729425 94.59422556544106
v -131.4513477496869 100.8892287208605 198.2552347090372
v 96.21634456247315 -45.762988